In [1]:
import glob
# glob 함수는 사용자가 제시한 조건에 맞는 파일명을 리스트 형식으로 반환
import os, re
import tensorflow as tf
import numpy as np

In [2]:
txt_dir = r"C:\Users\Jennie\Desktop\aiffel\Aiffel_MiniProject6\Jennie_Aiffel\lyrics\*"
txt_list = glob.glob(txt_dir)

raw_corpus = []
for txt_file in txt_list:
    with open(txt_file, "r", encoding="utf-8") as f:
        raw = f.read(). splitlines()
        raw_corpus.extend(raw) #추가 내용 연장
        # append, extend차이 https://m.blog.naver.com/wideeyed/221541104629
print("데이터 크기: ", len(raw_corpus))

데이터 크기:  187088


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() 
    sentence = re.sub(r"([.,¿])", r" \1", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence) 
    sentence = re.sub(r"[;:@#$%&*.,¿]+", " ", sentence) 
    sentence = sentence.strip()
    sentence = '<start> ' + sentence + ' <end>'
    return sentence
  
print(preprocess_sentence("This isn't ;;;sample    sentence."))   

<start> this isn't  sample sentence <end>


In [4]:
# 정제 문장 모으기
corpus = []
# 정제안된 코퍼스 리스트에 저장된 문장들을 순서대로 반환하여 setence에 저장
for sentence in raw_corpus:
    if len(sentence)==0: continue
    if sentence[-1]==":": continue
    # 앞서 구현한 정제 함수를 이용하여 문장 정제하고 담기
    pre_sentence = preprocess_sentence(sentence)
    corpus.append(pre_sentence)

print(corpus[:5])
print(len(corpus))

['<start> looking for some education <end>', '<start> made my way into the night <end>', '<start> all that bullshit conversation <end>', "<start> baby   can't you read the signs? i won't bore you with the details   baby <end>", "<start> i don't even wanna waste your time <end>"]
175749


In [5]:
# 토큰화할때 텐서플로우의 Tokenizer와 pad_sequences를 사용
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=14000, 
        filters=' ',
        oov_token="<unk>"
    )
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)
    print("토큰 개수 15개 이상 처리 전:", len(tensor))
    for word in tensor:
        if len(word)>=15:
            tensor.remove(word)
    print("토큰 개수 15개 이상 처리 후:", len(tensor))
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

토큰 개수 15개 이상 처리 전: 175749
토큰 개수 15개 이상 처리 후: 160892
[[  2 298  23 ...   0   0   0]
 [  2 217  11 ...   0   0   0]
 [  2  21  14 ...   0   0   0]
 ...
 [  2  24  67 ...   0   0   0]
 [  2  35  20 ...   0   0   0]
 [  2  24  67 ...   0   0   0]] <keras.preprocessing.text.Tokenizer object at 0x000002565A38C130>


In [7]:
from sklearn.model_selection import train_test_split

# 자연어처리 분야에서 모델의 입력이 되는 문장을 src ,
# 정답 역할을 하게 될 모델의 출력 문장을 타겟 문장 tgt

src_input = tensor[:, :-1]
print("source sentence: ", src_input.shape)
print(src_input[0])
tgt_input = tensor[:, 1:]
print("target sentence: ", tgt_input.shape)
print(tgt_input[0])

# tokenize() 함수로 데이터를 Tensor로 변환한 후,
# sklearn 모듈의 train_test_split() 함수를 사용해
# 훈련 데이터와 평가 데이터를 분리
# 단어장의 크기는 12,000 이상 으로 설정
# 총 데이터의 20% 를 평가 데이터셋으로 사용하기
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, shuffle=True, random_state=42)
print(enc_train.shape)
print(enc_val.shape)
print(dec_train.shape)
print(dec_val.shape)

source sentence:  (160892, 14)
[   2  298   23   88 5223    3    0    0    0    0    0    0    0    0]
target sentence:  (160892, 14)
[ 298   23   88 5223    3    0    0    0    0    0    0    0    0    0]
(128713, 14)
(32179, 14)
(128713, 14)
(32179, 14)


In [8]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        # Embedding layer, 2개 LSTM layer, 1개 Dense layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear1 = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        out = self.embedding(x)
        out = self.rnn1(out)
        out = self.rnn2(out)
        out = self.linear1(out)
        return out

In [9]:
embedding_size = 1024
hidden_size = 2048 
vocab_size = tokenizer.num_words+1
print(vocab_size)
model = TextGenerator(vocab_size, embedding_size, hidden_size)

14001


In [10]:
for idx in tokenizer.index_word:
  print(idx, ":", tokenizer.index_word[idx])
  if idx>=10: break

1 : <unk>
2 : <start>
3 : <end>
4 : the
5 : i
6 : you
7 : and
8 : to
9 : a
10 : me


In [11]:
load_status = model.load_weights("ckpt")

## 원본 문장생성기

In [19]:
import numpy as np

def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해서 문장 만들기
    while True:
        # 1. 입력받은 문장의 텐서를 입력합니다
        predict = model(test_tensor) 
        # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]
        # 3. 2에서 예측된 word index를 문장 뒤에 붙입니다
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
        # (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
        if predict_word.numpy()[0]==end_token: break
        if test_tensor.shape[1] >= max_len: break
    
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    return generated # 최종적으로 모델이 생성한 문장 반환

In [20]:
print(generate_text(model, tokenizer, init_sentence=" <start>Who"))

<unk> all night long we're gonna rock this house until we knock it down <end> 


In [21]:
print(generate_text(model, tokenizer, init_sentence=" <start> I love"))

<start> i love you <end> 


## 새로운 문장생성기

In [30]:
def generate_text2(model, tokenizer, init_sentence="<start>", max_len=15):
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해서 문장 만들기
    while True:
        # 1. 입력받은 문장의 텐서를 입력합니다
        predict = model(test_tensor) 
        # 2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
        predict = tf.nn.softmax(tf.squeeze(predict))
        p_predict = np.array(predict[0])
        p_predict /= p_predict.sum()
        predict_word = np.random.choice(len(p_predict), size=1, p=p_predict)
        predict_word = tf.convert_to_tensor(predict_word, dtype=tf.int64)
        # 3. 2에서 예측된 word index를 문장 뒤에 붙입니다
        expand_word = tf.expand_dims(predict_word, axis=0)
        test_tensor = tf.concat([test_tensor, expand_word], axis=-1)
        # 4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
        # (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
        if predict_word[0]==end_token: break
        if test_tensor.shape[1] >= max_len: break
    
    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "
    return generated # 최종적으로 모델이 생성한 문장 반환

In [31]:
print(generate_text2(model, tokenizer, init_sentence=" <start> I love "))

<start> i love ah if where every i (21st now but and <unk> please room 


In [32]:
print(generate_text2(model, tokenizer, init_sentence=" <start> I love "))

<start> i love the someone upon i one i'm lift and memphis you feeling there's 
